# Librerias

In [ ]:
%load_ext autoreload
%autoreload 2
import pickle

import pandas as pd
import univariate_utils
import multivariate_utils

from exploration import load_datasets
from model_pruning import (
    assign_dataset_configs,
    transform_datasets,
    remove_high_colineality_columns,
    train_base_dataset_models,
    train_best_features_models
)

# Constantes

In [ ]:
ORIGINAL_METRICS_FILE = "results/summary.csv"
EXTENDED_METRICS_FILE = "extended_data_results/summary.csv"

In [ ]:
OUTPUT_MODELS_FILE = "final_models/trained_no_colineal_models.pkl"

In [ ]:
NUM_TRANSFORMS_TO_IGNORE = ["powerTransformer", "polynomialFeatures"]

In [ ]:
SELECTION_METRIC = "R2"
LOWER_IS_BEST = False

In [ ]:
MAX_COLINEALITY = 0.90

# Utils
Validando y seleccionando de los modelos más sobresalientes para cada conjunto de datos, basándose en una métrica particular y siguiendo criterios de selección predefinidos.

In [ ]:
def filter_best_models(df: pd.DataFrame) -> pd.DataFrame:
    """Filters best model config by Marca and selected metric in test"""
    df_test = df[
        df["split"] == "test"
    ].sort_values(SELECTION_METRIC, ascending=LOWER_IS_BEST)
    
    df_best = df_test.groupby("dataset").head(1).sort_values("dataset")
    
    return df_best

# Ejecucion

## Lectura de metricas

In [ ]:
df_original = pd.read_csv(ORIGINAL_METRICS_FILE)
df_extended = pd.read_csv(EXTENDED_METRICS_FILE)

### Ignorando polynomial features por dificultad de interpretacion
Se procede a eliminar del análisis ciertas transformaciones de datos en nuestros DataFrames, df_original y df_extended. Estas transformaciones específicas están definidas en una lista llamada NUM_TRANSFORMS_TO_IGNORE. La razón es para enfocarnos en un subconjunto específico de transformaciones que son más relevantes o más fáciles de interpretar en nuestro análisis. Al hacerlo, podemos simplificar nuestro conjunto de datos y centrarnos en lo que realmente importa para nuestro análisis. Esto nos ayuda a tener una comprensión más clara de los resultados y a tomar decisiones más informadas

In [ ]:
for trans in NUM_TRANSFORMS_TO_IGNORE:
    df_original = df_original[
        df_original["num_config"] != trans
    ].copy(deep=True)
    df_extended = df_extended[
        df_extended["num_config"] != trans
    ].copy(deep=True)    

In [ ]:
df_original.head(5)

In [ ]:
df_extended.head(5)

# Filtrado de mejores resultados en test 
El modelo ampliado demuestra consistentemente un mejor rendimiento por marca en comparación con el modelo que utiliza solo las características originales. Sin embargo, este aumento en el rendimiento también se acompaña de una complejidad adicional, ya que ambos modelos tienen un número significativo de características. Para facilitar la interpretación de los resultados, se hace necesaria la eliminación de características, lo que implica identificar y retener solo las características más relevantes y significativas para el análisis. Esta estrategia de selección de características es esencial para garantizar que el modelo sea interpretable y permita una comprensión más clara de la influencia de cada característica en las predicciones.

In [ ]:
df_original_best = filter_best_models(df_original)
df_original_best

In [ ]:
df_extended_best = filter_best_models(df_extended)
df_extended_best

In [ ]:
df_original_best["feature_set"] = "original"
df_extended_best["feature_set"] = "extended"

df_best_configs = filter_best_models(
    pd.concat([df_original_best, df_extended_best])
)

In [ ]:
df_best_configs["feature_count"] = df_best_configs["columns"].apply(
    lambda x: len(eval(x))
)

In [ ]:
df_best_configs

# Visualizando que influencia mas las metricas en test
Explorando cómo diferentes variables, como el conjunto de datos, la configuración numérica, las configuraciones escaladas y las codificadas, afectan una métrica específica que utilizamos para seleccionar modelos en nuestro conjunto de prueba. Verificando de esta manera, qué variables tienen un impacto más significativo en el rendimiento de nuestros modelos. Esta exploración nos ayuda a identificar las características que más influyen en la métrica de selección, lo que a su vez nos permite tomar decisiones más informadas para mejorar el rendimiento de nuestros modelos.

In [ ]:
df_plot_metrics = df_extended[
    df_extended["split"] == "test"
].copy(deep=True)

In [ ]:
df_plot_metrics["scat_config_type"] =  df_plot_metrics["scat_config"].apply(
    lambda x: x.split("__")[0]
)
df_plot_metrics["lcat_config_type"] =  df_plot_metrics["lcat_config"].apply(
    lambda x: x.split("__")[0]
)

In [ ]:
for col in ["dataset", "num_config", "scat_config", "lcat_config", "model"]:
    print("*" * 100)
    print(f"{col} influence over {SELECTION_METRIC}")
    multivariate_utils.plot_distributions(
        df_plot_metrics,
        SELECTION_METRIC,
        col,
        (10, 5),
        "box",  # Puede ser hist o box
        False
    )
    multivariate_utils.make_column_non_graphic_analysis(
        df_plot_metrics,
        SELECTION_METRIC,
        col
    )

# Transformacion de datasets

In [ ]:
datasets = assign_dataset_configs(df_best_configs)

In [ ]:
datasets["muelle"].keys()

In [ ]:
for marca in datasets:
    print(marca)
    print(datasets[marca]["config"])

In [ ]:
datasets_trans = transform_datasets(datasets)

In [ ]:
(
    datasets_trans["muelle"]["X_train"].shape,
    datasets_trans["muelle"]["X_test"].shape,
    len(datasets_trans["muelle"]["columns"])
)

# Eliminacion de columnas con alta colinealidad

In [ ]:
remove_high_colineality_columns(datasets_trans, MAX_COLINEALITY)

In [ ]:
for marca, marca_config in datasets_trans.items():
    print(f"Marca {marca} now has {len(marca_config['columns'])} features")

# Entrenamiento de modelos para hallar features mas relevantes
Como se usan modelos no lineales, se necesita entender la linealidad de las features con la variable de salida por medio de la importancia de las features.

In [ ]:
datasets_trans["muelle"]["config"]

In [ ]:
train_base_dataset_models(datasets_trans)

In [ ]:
datasets_trans["muelle"]["metrics"]

In [ ]:
df_no_colineal_metrics = pd.concat(
    [
        marca_config["metrics"]
        for marca, marca_config in datasets_trans.items()
    ]
)
df_no_colineal_metrics

In [ ]:
df_no_colineal_metrics[
    df_no_colineal_metrics["split"] == "test"
].sort_values("marca")

# Entrenamiento de modelos con features que altos coeficientes
Entrenamiento de los modelos para diferentes marcas en el conjunto de datos. Primero, se seleccionan las 20 características principales para cada marca en el conjunto de datos. Luego, se entrenan modelos utilizando estas características seleccionadas. Se realiza un seguimiento de las métricas de rendimiento de cada marca y se muestran las columnas seleccionadas para cada modelo. Finalmente, los modelos resultantes se guardan en el archivo utilizando el módulo de Python "pickle". Entrenamiento, seguimiento y almacenamiento de modelos basados en las mejores características para diferentes marcas en el conjunto de datos.

In [ ]:
MARCA_TOP_FEATURES = {
    marca: 20
    for marca in datasets_trans
}

In [ ]:
top_features_datasets_trans = train_best_features_models(datasets_trans, MARCA_TOP_FEATURES)

In [ ]:
train_base_dataset_models(top_features_datasets_trans)

In [ ]:
for marca, marca_config in top_features_datasets_trans.items():
    print(f"Marca {marca} now has {len(marca_config['columns'])} features")

In [ ]:
df_top_feat_metrics = pd.concat(
    [
        marca_config["metrics"]
        for marca, marca_config in top_features_datasets_trans.items()
    ]
)
df_top_feat_metrics

In [ ]:
df_top_feat_metrics[
    df_top_feat_metrics["split"] == "test"
].sort_values("marca")

In [ ]:
for marca, marca_config in top_features_datasets_trans.items():
    print("*" * 20)
    print(f"Marca {marca} columns: {marca_config['columns']}")

In [ ]:
with open(OUTPUT_MODELS_FILE, "wb") as f:
    pickle.dump(top_features_datasets_trans, f)